THIS IS THE NOTEBEOOK WHERE WE ARE GOING TO FIRST INITIALIZE A MODEL AND THEN THEN EMBEDD THE DATA INTO THE RAG PIPELINE


In [1]:
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig

In [3]:
llm = ChatOllama(
    model="llama3.2:3b",  # or any local ollama model name you’ve pulled
    temperature=0.2,
)

checkpointer = InMemorySaver()

In [4]:
agent = create_agent(
    model=llm,          # ⬅️ pass the ChatOllama instance, NOT a string
    tools=[],           # add tools here if you have any
    checkpointer=checkpointer,
)

In [5]:
import pandas as pd
from langchain_core.documents import Document

# Load the FIFA matches data
df = pd.read_csv("Fifa_world_cup_matches.csv")  # adjust path if needed

# Quick peek
print(df.head())
print(df.columns)


           team1         team2 possession team1 possession team2  \
0          QATAR       ECUADOR              42%              50%   
1        ENGLAND          IRAN              72%              19%   
2        SENEGAL   NETHERLANDS              44%              45%   
3  UNITED STATES         WALES              51%              39%   
4      ARGENTINA  SAUDI ARABIA              64%              24%   

  possession in contest  number of goals team1  number of goals team2  \
0                    8%                      0                      2   
1                    9%                      6                      2   
2                   11%                      0                      2   
3                   10%                      1                      1   
4                   12%                      1                      2   

          date     hour category  ...  penalties scored team1  \
0  20 NOV 2022  17 : 00  Group A  ...                       0   
1  21 NOV 2022  14 : 0

converting each row of the data into text to embedded into the vector Store

In [6]:
df.columns


Index(['team1', 'team2', 'possession team1', 'possession team2',
       'possession in contest', 'number of goals team1',
       'number of goals team2', 'date', 'hour', 'category',
       'total attempts team1', 'total attempts team2', 'conceded team1',
       'conceded team2', 'goal inside the penalty area team1',
       'goal inside the penalty area team2',
       'goal outside the penalty area team1',
       'goal outside the penalty area team2', 'assists team1', 'assists team2',
       'on target attempts team1', 'on target attempts team2',
       'off target attempts team1', 'off target attempts team2',
       'attempts inside the penalty area team1',
       'attempts inside the penalty area  team2',
       'attempts outside the penalty area  team1',
       'attempts outside the penalty area  team2', 'left channel team1',
       'left channel team2', 'left inside channel team1',
       'left inside channel team2', 'central channel team1',
       'central channel team2', 'right in

In [7]:
df.columns

Index(['team1', 'team2', 'possession team1', 'possession team2',
       'possession in contest', 'number of goals team1',
       'number of goals team2', 'date', 'hour', 'category',
       'total attempts team1', 'total attempts team2', 'conceded team1',
       'conceded team2', 'goal inside the penalty area team1',
       'goal inside the penalty area team2',
       'goal outside the penalty area team1',
       'goal outside the penalty area team2', 'assists team1', 'assists team2',
       'on target attempts team1', 'on target attempts team2',
       'off target attempts team1', 'off target attempts team2',
       'attempts inside the penalty area team1',
       'attempts inside the penalty area  team2',
       'attempts outside the penalty area  team1',
       'attempts outside the penalty area  team2', 'left channel team1',
       'left channel team2', 'left inside channel team1',
       'left inside channel team2', 'central channel team1',
       'central channel team2', 'right in

In [12]:
def row_to_text(row):
    # Adjust column names to your actual CSV headers
    return (
       
        f"FIFA World Cup 2022 Match {row['team1']} vs {row['team2']}. "
        f"Score: {row['number of goals team1']} - {row['number of goals team2']}. "
        f"Date: {row['date']}. "
        f"Category{row['category']}"
        f"Possesion of {row['team1']} is {row['possession team1']} and {row['team2']} is {row['possession team2']} "
        f"{row['team1']} had {row['on target attempts team1']} On Target Attempts and {row['team2']} had  {row['on target attempts team2']} attempts"
         f"{row['team1']} had {row['assists team1']} assists and {row['team2']} had {row['assists team2']} assists"
    )

docs = []
for _, row in df.iterrows():
    text = row_to_text(row)
    metadata = {
        "team1": row["team1"],
        "team2": row["team2"],
        "category":row['category']
    }
    docs.append(Document(page_content=text, metadata=metadata))

len(docs), docs[0]


(64,
 Document(metadata={'team1': 'QATAR', 'team2': 'ECUADOR', 'category': 'Group A'}, page_content='FIFA World Cup 2022 Match QATAR vs ECUADOR. Score: 0 - 2. Date: 20 NOV 2022. CategoryGroup APossesion of QATAR is 42% and ECUADOR is 50% QATAR had 0 On Target Attempts and ECUADOR had  3 attemptsQATAR had 0 assists and ECUADOR had 1 assists'))

In [13]:
print(docs)

[Document(metadata={'team1': 'QATAR', 'team2': 'ECUADOR', 'category': 'Group A'}, page_content='FIFA World Cup 2022 Match QATAR vs ECUADOR. Score: 0 - 2. Date: 20 NOV 2022. CategoryGroup APossesion of QATAR is 42% and ECUADOR is 50% QATAR had 0 On Target Attempts and ECUADOR had  3 attemptsQATAR had 0 assists and ECUADOR had 1 assists'), Document(metadata={'team1': 'ENGLAND', 'team2': 'IRAN', 'category': 'Group B'}, page_content='FIFA World Cup 2022 Match ENGLAND vs IRAN. Score: 6 - 2. Date: 21 NOV 2022. CategoryGroup BPossesion of ENGLAND is 72% and IRAN is 19% ENGLAND had 7 On Target Attempts and IRAN had  3 attemptsENGLAND had 6 assists and IRAN had 1 assists'), Document(metadata={'team1': 'SENEGAL', 'team2': 'NETHERLANDS', 'category': 'Group A'}, page_content='FIFA World Cup 2022 Match SENEGAL vs NETHERLANDS. Score: 0 - 2. Date: 21 NOV 2022. CategoryGroup APossesion of SENEGAL is 44% and NETHERLANDS is 45% SENEGAL had 3 On Target Attempts and NETHERLANDS had  3 attemptsSENEGAL had 